In [ ]:
import os
import sys

from astropy.coordinates import SkyCoord
from astropy import units as u
from astropy.table import Table, unique, vstack, join, Column
from surface_brightness import surface_brightness


In [ ]:
#Input / Output Information
fits_directory = '/Volumes/Media RED/fits files/Friedman 15 Supernova' #The directory to look for .fits files in
output_file_name = "friedman 2015" #The output file name without an extension


In [ ]:
if os.path.exists(fits_directory) and os.listdir(fits_directory):
    cord_dict = {} #for supernova coordinates
    red_dict = {}#for supernova redshifts
    
    friedman_15_table = Table.read('../Friedman 2015/friedman 2015 data table.csv', format = 'csv')
    for row in friedman_15_table:
        red_dict[row['SN']] = row['z']
        cord_dict[row['SN']] = SkyCoord(ra = row['RAj2000'] * u.degree, dec = row['DEj2000'] * u.degree)
    
    brightness = surface_brightness(cord_dict, red_dict)
    
    #Individual tables are generated for nuv and fuv using both a 1kpc and 5kpc aperture
    print('Calculating NUV values using a 1kpc radius aperture\n', flush = True)
    nuv_1kpc = brightness.create_tables('NUV', fits_directory, 1, show_list = True)
    
    print('\nCalculating FUV values using a 1kpc radius aperture\n', flush = True)
    fuv_1kpc = brightness.create_tables('FUV', fits_directory, 1, show_list = True)
    
    print('\nCalculating NUV values using a 5kpc radius aperture\n', flush = True)
    nuv_5kpc = brightness.create_tables('NUV', fits_directory, 5, show_list = True)
    
    print('\nCalculating FUV values using a 5kpc radius aperture\n', flush = True)
    fuv_5kpc = brightness.create_tables('FUV', fits_directory, 5, show_list = True)
    
    #Keep a copy of the original tables in memory
        #This makes it faster to re-run the 
        #upcoming code if necessary
    nuv_1kpc_copy = nuv_1kpc
    fuv_1kpc_copy = fuv_1kpc
    nuv_5kpc_copy = nuv_5kpc
    fuv_5kpc_copy = fuv_5kpc
    
    #Create plots and save them to file
    print('Creating Plots\n', flush = True)
    brightness.create_plots(nuv_1kpc[0], 'NUV', 1)
    brightness.create_plots(fuv_1kpc[0], 'FUV', 1)
    
    #Create a table with the information desired to be in the .csv
    print('Joining Tables\n', flush = True)
    nuv_1kpc[0].keep_columns(['sn',
                           'NUV 1kpc Luminosity',
                           'NUV 1kpc Luminosity Error',
                           'NUV 1kpc Surface Brightness',
                           'NUV 1kpc Surface Brightness Error'])
    
    fuv_1kpc[0].keep_columns(['sn',
                           'FUV 1kpc Luminosity',
                           'FUV 1kpc Luminosity Error',
                           'FUV 1kpc Surface Brightness',
                           'FUV 1kpc Surface Brightness Error'])
    
    nuv_5kpc[0].remove_columns(['Flux',
                                'Flux Error',
                                'NUV 5kpc Luminosity',
                                'NUV 5kpc Luminosity Error'])
    
    fuv_5kpc[0].remove_columns(['Flux',
                                'Flux Error',
                                'FUV 5kpc Luminosity',
                                'FUV 5kpc Luminosity Error'])
    
    merged_nuv_table = join(nuv_1kpc[0], nuv_5kpc[0], join_type = 'outer', keys = 'sn')
    merged_fuv_table = join(fuv_1kpc[0], fuv_5kpc[0], join_type = 'outer', keys = 'sn')
    
    merged_table = join(merged_nuv_table, merged_fuv_table, join_type = 'outer', keys = ['sn'])
    merged_table.rename_column('NUV 5kpc Exposure Time', 'NUV Exposure Time')
    merged_table.rename_column('FUV 5kpc Exposure Time', 'FUV Exposure Time')
    merged_table.rename_column('Redshift_1', 'Redshift')
    merged_table.rename_column('Redshift Error_1', 'Redshift Error')
    
    for row in merged_table:
        if str(row["Redshift"]) == "--":
            row["Redshift"] = row["Redshift_2"]
            row["Redshift Error"] = row["Redshift Error_2"]
    
    merged_table.remove_columns(["Redshift_2", "Redshift Error_2"])
    
    #rearrange the column order
    final_table = merged_table['sn', 'Redshift', 'Redshift Error',
                               'NUV Exposure Time',
                               'NUV 1kpc Luminosity',
                               'NUV 1kpc Luminosity Error',
                               'NUV 1kpc Surface Brightness',
                               'NUV 1kpc Surface Brightness Error',
                               'NUV 5kpc Surface Brightness',
                               'NUV 5kpc Surface Brightness Error',
                               'FUV Exposure Time',
                               'FUV 1kpc Luminosity',
                               'FUV 1kpc Luminosity Error',
                               'FUV 1kpc Surface Brightness',
                               'FUV 1kpc Surface Brightness Error',
                               'FUV 5kpc Surface Brightness',
                               'FUV 5kpc Surface Brightness Error']
    
    final_table.sort('sn')
    
    merged_log = vstack([nuv_5kpc[1], fuv_5kpc[1]])
    merged_log.sort('File Path')
    
    #Write data to an output file
        #The csv format does not have an 'overwrite' kwarg,
        #but an existing file will be overwritten
    final_table.write(output_file_name + '.fits', overwrite = True, format = 'fits')
    final_table.write(output_file_name + '.csv', format = 'csv')
    merged_log.write(output_file_name + ' log.csv', format = 'csv')
    print('Finished')
    
elif not os.path.exists(fits_directory):
    print('Specified directory of .fits files does not exist')
    
else:
    print('Specified directory of .fits files is empty')
